# Two-Stage Training: Severstal Pre-training → Galicia Fine-tuning

**Fase 1**: Pre-entrenar con Severstal (6,666 imágenes) para aprender patrones genéricos de defectos en acero  
**Fase 2**: Fine-tune con Galicia (14 imágenes) para especializar en defectos específicos

**Respuesta a tu pregunta**: UNet++ con ResNet34 pre-entrenado **SÍ necesita 3 canales RGB** (ImageNet). Las imágenes de Galicia son RGB, así que no hay problema.

In [2]:
!pip install segmentation-models-pytorch albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.5 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# URL del dataset
zip_url = 'https://huggingface.co/datasets/rohanath/severstal-steel-detection/resolve/main/Severstal%20Steel%20Defect%20Detection.zip'



# 1. Descargar el ZIP (lo descargamos temporalmente en /content)
print("Descargando archivo ZIP...")
!wget -O severstal_steel_detection.zip $zip_url


Descargando archivo ZIP...
--2025-12-07 19:30:45--  https://huggingface.co/datasets/rohanath/severstal-steel-detection/resolve/main/Severstal%20Steel%20Defect%20Detection.zip
Resolving huggingface.co (huggingface.co)... 3.171.171.6, 3.171.171.128, 3.171.171.65, ...
Connecting to huggingface.co (huggingface.co)|3.171.171.6|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/68974552a4dbfa7414d73ca6/ad86142be80c3402759952836d79c2a0a8e43af08ca714ae6724b4eaf648fedf?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Severstal%252520Steel%252520Defect%252520Detection.zip%3B+filename%3D%22Severstal%2520Steel%2520Defect%2520Detection.zip%22%3B&response-content-type=application%2Fzip&Expires=1765139445&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY1MTM5NDQ1fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjg5NzQ1NTJhNGRiZmE3NDE0ZDczY2E2L2FkODYxNDJiZTgwYzM0M

In [5]:

# Ruta donde se guardará el dataset DENTRO de tu Drive
drive_path = '/content/drive/MyDrive/severstal'
!mkdir -p "$drive_path"

# 4. Descomprimir el archivo directamente en la ruta de Drive
print(f"Descomprimiendo en: {drive_path}...")
!unzip severstal_steel_detection.zip -d $drive_path

Se han truncado las últimas 5000 líneas del flujo de salida.
 extracting: /content/drive/MyDrive/severstal/severstal-steel-defect-detection/val/labels/89c28225d.txt  
  inflating: /content/drive/MyDrive/severstal/__MACOSX/severstal-steel-defect-detection/val/labels/._89c28225d.txt  
 extracting: /content/drive/MyDrive/severstal/severstal-steel-defect-detection/val/labels/4a00eaee7.txt  
  inflating: /content/drive/MyDrive/severstal/__MACOSX/severstal-steel-defect-detection/val/labels/._4a00eaee7.txt  
 extracting: /content/drive/MyDrive/severstal/severstal-steel-defect-detection/val/labels/7ec438623.txt  
  inflating: /content/drive/MyDrive/severstal/__MACOSX/severstal-steel-defect-detection/val/labels/._7ec438623.txt  
 extracting: /content/drive/MyDrive/severstal/severstal-steel-defect-detection/val/labels/1aa7e1c47.txt  
  inflating: /content/drive/MyDrive/severstal/__MACOSX/severstal-steel-defect-detection/val/labels/._1aa7e1c47.txt  
 extracting: /content/drive/MyDrive/severstal/s

In [6]:
# Ruta donde se guardará el dataset DENTRO de tu Drive
drive_path = '/content/drive/MyDrive/unet_dataset'
!mkdir -p "$drive_path"

# 4. Descomprimir el archivo directamente en la ruta de Drive
print(f"Descomprimiendo en: {drive_path}...")
!unzip /content/drive/MyDrive/datasets.zip -d $drive_path

Descomprimiendo en: /content/drive/MyDrive/unet_dataset...
Archive:  /content/drive/MyDrive/datasets.zip
   creating: /content/drive/MyDrive/unet_dataset/datasets/
   creating: /content/drive/MyDrive/unet_dataset/datasets/unet_dataset/
   creating: /content/drive/MyDrive/unet_dataset/datasets/unet_dataset/train/
   creating: /content/drive/MyDrive/unet_dataset/datasets/unet_dataset/train/images/
  inflating: /content/drive/MyDrive/unet_dataset/datasets/unet_dataset/train/images/imagenes_galicia_19.jpg  
  inflating: /content/drive/MyDrive/unet_dataset/datasets/unet_dataset/train/images/raw_image_toshiba_50.jpg  
  inflating: /content/drive/MyDrive/unet_dataset/datasets/unet_dataset/train/images/imagenes_galicia_26.jpg  
  inflating: /content/drive/MyDrive/unet_dataset/datasets/unet_dataset/train/images/raw_image_toshiba_25.jpg  
  inflating: /content/drive/MyDrive/unet_dataset/datasets/unet_dataset/train/images/raw_image_toshiba_44.jpg  
  inflating: /content/drive/MyDrive/unet_dataset

In [35]:
# ==========================================
# IMPORTS
# ==========================================
import os
import random
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from torch.cuda.amp import autocast, GradScaler # <--- ¡IMPORTANTE PARA A100!
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


# Configuración del Scaler para Mixed Precision
scaler = GradScaler()

print(f"✅ Librerías cargadas. AMP activado para {torch.cuda.get_device_name(0)}")

PyTorch: 2.9.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
✅ Librerías cargadas. AMP activado para NVIDIA A100-SXM4-40GB


/tmp/ipython-input-1776448026.py:28: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [8]:
# ==========================================
# CONFIGURACIÓN GLOBAL
# ==========================================
SEED = 42
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Rutas
SEVERSTAL_ROOT = '/content/drive/MyDrive/severstal/severstal-steel-defect-detection'
GALICIA_ROOT = '/content/drive/MyDrive/unet_dataset'

# Fase 1: Severstal Pre-training
PHASE1_EPOCHS = 30
PHASE1_BATCH = 16
PHASE1_LR = 1e-4
PHASE1_SIZE = 256  # Severstal images son 1600x256, resize a 256x256

# Fase 2: Galicia Fine-tuning
PHASE2_EPOCHS = 100
PHASE2_BATCH = 4
PHASE2_LR = 1e-5  # LR más bajo para fine-tuning
PHASE2_SIZE = 512  # Tiles de Galicia

# Modelo
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'

# Seeds
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"Device: {DEVICE}")
print(f"Severstal: {PHASE1_EPOCHS} epochs, batch {PHASE1_BATCH}, LR {PHASE1_LR}")
print(f"Galicia: {PHASE2_EPOCHS} epochs, batch {PHASE2_BATCH}, LR {PHASE2_LR}")

Device: cuda
Severstal: 30 epochs, batch 16, LR 0.0001
Galicia: 100 epochs, batch 4, LR 1e-05


---
## 📦 PARTE 1: Dataset Severstal (RLE Masks)
---

In [9]:
# ==========================================
# RLE DECODER (Severstal usa Run-Length Encoding)
# ==========================================
def rle_decode(mask_rle, shape=(256, 1600)):
    """
    Decode RLE string to binary mask
    mask_rle: RLE string (e.g., '1 3 10 5' means pixel 1-3, 10-14)
    shape: (height, width) de la imagen original
    """
    if pd.isna(mask_rle) or mask_rle == '':
        return np.zeros(shape, dtype=np.uint8)

    s = mask_rle.split()
    starts = np.array([int(s[i]) for i in range(0, len(s), 2)]) - 1  # RLE es 1-indexed
    lengths = np.array([int(s[i]) for i in range(1, len(s), 2)])
    ends = starts + lengths

    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for start, end in zip(starts, ends):
        img[start:end] = 1

    return img.reshape(shape, order='F')  # Fortran order (column-major)

# Test
test_mask = rle_decode('1 3 10 5', shape=(10, 10))
print(f"RLE test shape: {test_mask.shape}, non-zero: {test_mask.sum()}")

RLE test shape: (10, 10), non-zero: 8


In [27]:
# ==========================================
# PREPARAR DATOS SEVERSTAL
# ==========================================
df_train = pd.read_csv(os.path.join(SEVERSTAL_ROOT, 'train.csv'))
print(f"Total rows en train.csv: {len(df_train)}")
print(df_train.head())
# Limpiar nombres de columnas (eliminar espacios en blanco)
df_train.columns = df_train.columns.str.strip()
# Renombrar columnas
df_train = df_train.rename(columns={'ImageId': 'ImageID', 'ClassId': 'ClassID'})
# IMPORTANTE: Limpiar espacios en los valores de ImageID también
df_train['ImageID'] = df_train['ImageID'].str.strip()
# Convertir ClassID a int
df_train['ClassID'] = df_train['ClassID'].astype(int)
# Obtener imágenes únicas del CSV
img_ids_csv = df_train['ImageID'].unique()
print(f"Total imágenes únicas en CSV: {len(img_ids_csv)}")
# Filtrar solo las imágenes que existen en el directorio
img_dir = os.path.join(SEVERSTAL_ROOT, 'train', 'images')
existing_imgs = set(os.listdir(img_dir))
img_ids = [img_id for img_id in img_ids_csv if img_id in existing_imgs]
print(f"Total imágenes que existen en disco: {len(img_ids)}")
# Filtrar el dataframe para solo incluir imágenes existentes
df_train = df_train[df_train['ImageID'].isin(img_ids)]
print(f"Total rows después de filtrar: {len(df_train)}")
# Split train/val
train_ids, val_ids = train_test_split(img_ids, test_size=0.15, random_state=SEED)
print(f"Train: {len(train_ids)}, Val: {len(val_ids)}")

Total rows en train.csv: 7095
   ImageId         ClassId                                       EncodedPixels
0  0002cc93b.jpg          1  29102 12 29346 24 29602 24 29858 24 30114 24 3...
1  0007a71bf.jpg          3  18661 28 18863 82 19091 110 19347 110 19603 11...
2  000a4bcdd.jpg          1  37607 3 37858 8 38108 14 38359 20 38610 25 388...
3  000f6bf48.jpg          4  131973 1 132228 4 132483 6 132738 8 132993 11 ...
4  0014fce06.jpg          3  229501 11 229741 33 229981 55 230221 77 230468...
Total imágenes únicas en CSV: 6666
Total imágenes que existen en disco: 6666
Total rows después de filtrar: 7095
Train: 5666, Val: 1000


In [29]:
# ==========================================
# DATASET SEVERSTAL (SIN RESIZE DESTRUCTIVO)
# ==========================================
class SeverStalDataset(Dataset):
    def __init__(self, img_ids, df, img_dir, transform=None):
        self.img_ids = img_ids
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_path = os.path.join(self.img_dir, img_id)

        # Leer imagen
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # La imagen original es 256x1600

        # Crear máscara (shape original: 256x1600)
        mask = np.zeros((256, 1600), dtype=np.uint8)

        # Rellenar máscara
        img_df = self.df[self.df['ImageID'] == img_id]
        for _, row in img_df.iterrows():
            class_id = int(row['ClassID'])
            rle = row['EncodedPixels']
            if pd.notna(rle):
                class_mask = rle_decode(rle, shape=(256, 1600))
                mask[class_mask == 1] = class_id

        # --- AQUI ESTABA EL ERROR: ELIMINAMOS EL RESIZE MANUAL ---
        # Dejamos que 'transform' (Albumentations) haga el Crop

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.long()

# Re-crear los DataLoaders con el nuevo Dataset y Transform
train_ds_severstal = SeverStalDataset(train_ids, df_train, os.path.join(SEVERSTAL_ROOT, 'train/images'), transform=train_transform_phase1)
val_ds_severstal = SeverStalDataset(val_ids, df_train, os.path.join(SEVERSTAL_ROOT, 'train/images'), transform=val_transform_phase1)

# Aumentamos batch_size a 32 porque AMP ahorra memoria (si te da error de memoria, bájalo a 16)
BATCH_SIZE_AMP = 32
train_loader_phase1 = DataLoader(train_ds_severstal, batch_size=BATCH_SIZE_AMP, shuffle=True, num_workers=2, pin_memory=True)
val_loader_phase1 = DataLoader(val_ds_severstal, batch_size=BATCH_SIZE_AMP, shuffle=False, num_workers=2, pin_memory=True)

print("✅ DataLoaders actualizados.")

Directorio de imágenes: /content/drive/MyDrive/severstal/severstal-steel-defect-detection/train/images
¿El directorio existe? True
Primeros 5 archivos: ['58ee62fd7.jpg', 'eeffa4c49.jpg', 'e2bdd4236.jpg', '5210ac609.jpg', '55901ac51.jpg']
Severstal sample - Image: torch.Size([3, 256, 256]), Mask: torch.Size([256, 256]), Classes: [0 3]


---
## 📦 PARTE 2: Dataset Galicia (PNG Masks)
---

In [30]:
# ==========================================
# SMART SAMPLING DATASET GALICIA
# ==========================================
class GaliciaSmartDataset(Dataset):
    """
    Dataset que escanea máscaras y prioriza patches con defectos
    60% defectos, 40% background
    """
    def __init__(self, root_dir, transform=None, tile_size=512, defect_ratio=0.6):
        self.root_dir = root_dir
        self.transform = transform
        self.tile_size = tile_size
        self.defect_ratio = defect_ratio

        self.img_dir = os.path.join(root_dir, 'images')
        self.mask_dir = os.path.join(root_dir, 'masks')
        self.images = sorted([f for f in os.listdir(self.img_dir) if f.endswith('.jpg')])

        # Escanear máscaras y encontrar patches con defectos
        print("🔍 Escaneando dataset Galicia...")
        self.defect_patches = []
        self.background_patches = []

        for img_name in tqdm(self.images):
            mask_name = img_name.replace('.jpg', '.png')
            mask_path = os.path.join(self.mask_dir, mask_name)

            if not os.path.exists(mask_path):
                continue

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            h, w = mask.shape

            # Extraer múltiples patches por imagen
            for _ in range(20):  # 20 patches por imagen
                if np.random.rand() < self.defect_ratio:
                    # Buscar patch con defecto
                    ys, xs = np.where(mask > 0)
                    if len(ys) > 0:
                        idx = np.random.randint(len(ys))
                        cy, cx = ys[idx], xs[idx]
                        y1 = max(0, min(cy - tile_size//2, h - tile_size))
                        x1 = max(0, min(cx - tile_size//2, w - tile_size))
                        self.defect_patches.append((img_name, y1, x1))
                else:
                    # Patch aleatorio (background)
                    y1 = np.random.randint(0, max(1, h - tile_size))
                    x1 = np.random.randint(0, max(1, w - tile_size))
                    self.background_patches.append((img_name, y1, x1))

        print(f"✅ Patches con defectos: {len(self.defect_patches)}")
        print(f"✅ Patches background: {len(self.background_patches)}")

        # Combinar
        self.all_patches = self.defect_patches + self.background_patches
        random.shuffle(self.all_patches)

    def __len__(self):
        return len(self.all_patches)

    def __getitem__(self, idx):
        img_name, y1, x1 = self.all_patches[idx]

        img_path = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name.replace('.jpg', '.png'))

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Crop patch
        image = image[y1:y1+self.tile_size, x1:x1+self.tile_size]
        mask = mask[y1:y1+self.tile_size, x1:x1+self.tile_size]

        # Augmentations
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.long()

---
## 🔧 FUNCIONES DE ENTRENAMIENTO
---

In [31]:
# ==========================================
# MÉTRICAS
# ==========================================
def compute_iou(pred, target, num_classes):
    """
    Compute IoU per class
    """
    ious = []
    pred = pred.cpu().numpy()
    target = target.cpu().numpy()

    for cls in range(num_classes):
        pred_cls = (pred == cls)
        target_cls = (target == cls)

        intersection = np.logical_and(pred_cls, target_cls).sum()
        union = np.logical_or(pred_cls, target_cls).sum()

        if union == 0:
            ious.append(float('nan'))  # Clase no presente
        else:
            ious.append(intersection / union)

    return ious

# ==========================================
# TRAINING LOOP
# ==========================================
def train_one_epoch(model, loader, criterion, optimizer, device, num_classes):
    model.train()
    total_loss = 0
    all_ious = []

    for images, masks in tqdm(loader, desc='Training'):
        images = images.to(device).float()
        masks = masks.to(device).long()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # IoU
        preds = outputs.argmax(dim=1)
        batch_ious = compute_iou(preds, masks, num_classes)
        all_ious.append(batch_ious)

    avg_loss = total_loss / len(loader)
    avg_ious = np.nanmean(all_ious, axis=0)
    mean_iou = np.nanmean(avg_ious)

    return avg_loss, mean_iou, avg_ious

# ==========================================
# VALIDATION LOOP
# ==========================================
@torch.no_grad()
def validate(model, loader, criterion, device, num_classes):
    model.eval()
    total_loss = 0
    all_ious = []

    for images, masks in tqdm(loader, desc='Validation'):
        images = images.to(device).float()
        masks = masks.to(device).long()

        outputs = model(images)
        loss = criterion(outputs, masks)
        total_loss += loss.item()

        preds = outputs.argmax(dim=1)
        batch_ious = compute_iou(preds, masks, num_classes)
        all_ious.append(batch_ious)

    avg_loss = total_loss / len(loader)
    avg_ious = np.nanmean(all_ious, axis=0)
    mean_iou = np.nanmean(avg_ious)

    return avg_loss, mean_iou, avg_ious

---
# 🚀 FASE 1: PRE-ENTRENAMIENTO CON SEVERSTAL
---

In [32]:
# ==========================================
# AUGMENTATIONS FASE 1 (CORREGIDO: CROP vs RESIZE)
# ==========================================
# Las imágenes originales son 1600x256.
# En lugar de aplastarlas a 256x256, recortamos un trozo aleatorio de 256x256.

train_transform_phase1 = A.Compose([
    A.RandomCrop(height=256, width=256, p=1.0), # <--- ESTA ES LA CLAVE
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5), # Añadido vertical flip, ayuda en acero
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(var_limit=(10, 50), p=0.3),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# Para validación, en lugar de random crop, podemos hacer un CenterCrop o
# (mejor aún) validar con crops deslizantes, pero por simplicidad usaremos CenterCrop aquí.
val_transform_phase1 = A.Compose([
    A.CenterCrop(height=256, width=256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# ==========================================
# DATASETS & DATALOADERS FASE 1
# ==========================================
train_ds_severstal = SeverStalDataset(
    train_ids, df_train,
    os.path.join(SEVERSTAL_ROOT, 'train/images'),
    transform=train_transform_phase1,
    img_size=PHASE1_SIZE
)

val_ds_severstal = SeverStalDataset(
    val_ids, df_train,
    os.path.join(SEVERSTAL_ROOT, 'train/images'),
    transform=val_transform_phase1,
    img_size=PHASE1_SIZE
)

train_loader_phase1 = DataLoader(train_ds_severstal, batch_size=PHASE1_BATCH,
                                  shuffle=True, num_workers=2, pin_memory=True)
val_loader_phase1 = DataLoader(val_ds_severstal, batch_size=PHASE1_BATCH,
                                shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ Severstal Train: {len(train_ds_severstal)} samples")
print(f"✅ Severstal Val: {len(val_ds_severstal)} samples")

✅ Severstal Train: 5666 samples
✅ Severstal Val: 1000 samples


/tmp/ipython-input-2844698996.py:7: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10, 50), p=0.3),


In [33]:
# ==========================================
# MODELO FASE 1 (5 clases: 0=bg + 4 defectos)
# ==========================================
NUM_CLASSES_PHASE1 = 5  # Background + 4 defectos

model_phase1 = smp.UnetPlusPlus(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    in_channels=3,
    classes=NUM_CLASSES_PHASE1
).to(DEVICE)

# Loss con class weights (Severstal está desbalanceado)
class_weights_phase1 = torch.tensor([0.5, 2.0, 2.0, 2.0, 2.0]).to(DEVICE)  # Penalizar defectos minoritarios
criterion_phase1 = nn.CrossEntropyLoss(weight=class_weights_phase1)

optimizer_phase1 = torch.optim.AdamW(model_phase1.parameters(), lr=PHASE1_LR)
scheduler_phase1 = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_phase1, mode='max', patience=5, factor=0.5
)

print(f"✅ Modelo UNet++ con {ENCODER} creado")
print(f"✅ Clases Fase 1: {NUM_CLASSES_PHASE1}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

✅ Modelo UNet++ con resnet34 creado
✅ Clases Fase 1: 5


In [34]:
# ==========================================
# ENTRENAMIENTO FASE 1
# ==========================================
print("\n" + "="*60)
print("🚀 FASE 1: ENTRENAMIENTO CON SEVERSTAL")
print("="*60 + "\n")

best_iou_phase1 = 0.0
history_phase1 = {'train_loss': [], 'val_loss': [], 'val_iou': []}

for epoch in range(PHASE1_EPOCHS):
    print(f"\nEpoch {epoch+1}/{PHASE1_EPOCHS}")
    print("-" * 60)

    # Train
    train_loss, train_iou, train_ious = train_one_epoch(
        model_phase1, train_loader_phase1, criterion_phase1,
        optimizer_phase1, DEVICE, NUM_CLASSES_PHASE1
    )

    # Val
    val_loss, val_iou, val_ious = validate(
        model_phase1, val_loader_phase1, criterion_phase1,
        DEVICE, NUM_CLASSES_PHASE1
    )

    # Scheduler
    scheduler_phase1.step(val_iou)
    current_lr = optimizer_phase1.param_groups[0]['lr']

    # Log
    history_phase1['train_loss'].append(train_loss)
    history_phase1['val_loss'].append(val_loss)
    history_phase1['val_iou'].append(val_iou)

    print(f"Train Loss: {train_loss:.4f} | Train IoU: {train_iou:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val IoU: {val_iou:.4f}")
    print(f"Val IoUs per class: {[f'{x:.3f}' for x in val_ious]}")
    print(f"LR: {current_lr}")

    # Save best
    if val_iou > best_iou_phase1:
        best_iou_phase1 = val_iou
        torch.save(model_phase1.state_dict(), 'severstal_best.pth')
        print(f"✅ Guardado modelo con IoU: {best_iou_phase1:.4f}")

print(f"\n{'='*60}")
print(f"🎯 FASE 1 COMPLETADA - Mejor IoU: {best_iou_phase1:.4f}")
print(f"{'='*60}\n")


🚀 FASE 1: ENTRENAMIENTO CON SEVERSTAL


Epoch 1/30
------------------------------------------------------------


Validation: 100%|██████████| 63/63 [00:07<00:00,  8.91it/s]


Train Loss: 0.7709 | Train IoU: 0.2440
Val Loss: 0.3661 | Val IoU: 0.3347
Val IoUs per class: ['0.930', '0.000', '0.000', '0.364', '0.379']
LR: 0.0001
✅ Guardado modelo con IoU: 0.3347

Epoch 2/30
------------------------------------------------------------


Validation: 100%|██████████| 63/63 [00:07<00:00,  8.92it/s]


Train Loss: 0.3824 | Train IoU: 0.3169
Val Loss: 0.2691 | Val IoU: 0.3614
Val IoUs per class: ['0.945', '0.000', '0.000', '0.419', '0.444']
LR: 0.0001
✅ Guardado modelo con IoU: 0.3614

Epoch 3/30
------------------------------------------------------------


...

Train Loss: 0.2084 | Train IoU: 0.4106
Val Loss: 0.1928 | Val IoU: 0.4247
Val IoUs per class: ['0.948', '0.215', '0.000', '0.472', '0.488']
LR: 5e-05

Epoch 18/30
------------------------------------------------------------


Validation: 100%|██████████| 63/63 [00:07<00:00,  8.83it/s]


Train Loss: 0.1951 | Train IoU: 0.4241
Val Loss: 0.1921 | Val IoU: 0.4437
Val IoUs per class: ['0.956', '0.245', '0.000', '0.501', '0.516']
LR: 5e-05
✅ Guardado modelo con IoU: 0.4437

Epoch 19/30
------------------------------------------------------------


Validation:  14%|█▍        | 9/63 [00:01<00:08,  6.69it/s]


KeyboardInterrupt: 

In [ ]:
# ==========================================
# TESTING FASE 1
# ==========================================
print("\n📊 TESTING FASE 1 (Severstal)")
print("-" * 60)

# Cargar mejor modelo
model_phase1.load_state_dict(torch.load('severstal_best.pth'))

test_loss, test_iou, test_ious = validate(
    model_phase1, val_loader_phase1, criterion_phase1,
    DEVICE, NUM_CLASSES_PHASE1
)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Mean IoU: {test_iou:.4f}")
print(f"Test IoU per class:")
for i, iou in enumerate(test_ious):
    print(f"  Class {i}: {iou:.4f}")

# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(history_phase1['train_loss'], label='Train Loss')
ax1.plot(history_phase1['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Severstal Training Loss')
ax1.legend()
ax1.grid(True)

ax2.plot(history_phase1['val_iou'], label='Val IoU', color='green')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('IoU')
ax2.set_title('Severstal Validation IoU')
ax2.legend()
ax2.grid(True)
plt.tight_layout()
plt.show()

---
# 🎯 FASE 2: FINE-TUNING CON GALICIA
---

In [ ]:
# ==========================================
# AUGMENTATIONS FASE 2 (MÁS AGRESIVOS)
# ==========================================
train_transform_phase2 = A.Compose([
    A.PadIfNeeded(min_height=PHASE2_SIZE, min_width=PHASE2_SIZE,
                  border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0),
    A.RandomCrop(height=PHASE2_SIZE, width=PHASE2_SIZE, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
    A.GaussNoise(var_limit=(10, 50), p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transform_phase2 = A.Compose([
    A.PadIfNeeded(min_height=PHASE2_SIZE, min_width=PHASE2_SIZE,
                  border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0),
    A.CenterCrop(height=PHASE2_SIZE, width=PHASE2_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# ==========================================
# DATASET GALICIA
# ==========================================
galicia_ds = GaliciaSmartDataset(
    GALICIA_ROOT,
    transform=train_transform_phase2,
    tile_size=PHASE2_SIZE
)

# Split train/val manualmente (80/20)
train_size = int(0.8 * len(galicia_ds))
val_size = len(galicia_ds) - train_size
train_ds_galicia, val_ds_galicia = torch.utils.data.random_split(
    galicia_ds, [train_size, val_size],
    generator=torch.Generator().manual_seed(SEED)
)

# Cambiar transform de validación
val_ds_galicia.dataset.transform = val_transform_phase2

train_loader_phase2 = DataLoader(train_ds_galicia, batch_size=PHASE2_BATCH,
                                  shuffle=True, num_workers=2, pin_memory=True)
val_loader_phase2 = DataLoader(val_ds_galicia, batch_size=PHASE2_BATCH,
                                shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ Galicia Train: {len(train_ds_galicia)} patches")
print(f"✅ Galicia Val: {len(val_ds_galicia)} patches")

In [ ]:
# ==========================================
# CARGAR PESOS DE FASE 1 Y ADAPTAR PARA FASE 2
# ==========================================
NUM_CLASSES_PHASE2 = 4  # Background + Pit + Scratch + Damage

# OPCIÓN 1: Rebuild model con nuevas clases
model_phase2 = smp.UnetPlusPlus(
    encoder_name=ENCODER,
    encoder_weights=None,  # NO cargar ImageNet, usaremos Severstal
    in_channels=3,
    classes=NUM_CLASSES_PHASE2
).to(DEVICE)

# Cargar encoder y decoder (excepto cabeza final)
print("📦 Cargando pesos de Severstal...")
pretrained_dict = torch.load('severstal_best.pth')

# Filtrar solo encoder/decoder (no cabeza final 'segmentation_head')
model_dict = model_phase2.state_dict()
pretrained_dict_filtered = {
    k: v for k, v in pretrained_dict.items()
    if k in model_dict and 'segmentation_head' not in k
}

model_dict.update(pretrained_dict_filtered)
model_phase2.load_state_dict(model_dict)
print(f"✅ Transferidos {len(pretrained_dict_filtered)} parámetros desde Severstal")

# Class weights para Galicia (ajustar según tu análisis)
class_weights_phase2 = torch.tensor([1.0, 5.0, 5.0, 10.0]).to(DEVICE)  # Damage es muy raro
criterion_phase2 = nn.CrossEntropyLoss(weight=class_weights_phase2)

# Optimizer con LR bajo
optimizer_phase2 = torch.optim.AdamW(model_phase2.parameters(), lr=PHASE2_LR)
scheduler_phase2 = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_phase2, mode='max', patience=10, factor=0.5
)

print(f"✅ Modelo adaptado para {NUM_CLASSES_PHASE2} clases (Galicia)")

In [ ]:
# ==========================================
# ENTRENAMIENTO FASE 2
# ==========================================
print("\n" + "="*60)
print("🎯 FASE 2: FINE-TUNING CON GALICIA")
print("="*60 + "\n")

best_iou_phase2 = 0.0
history_phase2 = {'train_loss': [], 'val_loss': [], 'val_iou': []}

for epoch in range(PHASE2_EPOCHS):
    print(f"\nEpoch {epoch+1}/{PHASE2_EPOCHS}")
    print("-" * 60)

    # Train
    train_loss, train_iou, train_ious = train_one_epoch(
        model_phase2, train_loader_phase2, criterion_phase2,
        optimizer_phase2, DEVICE, NUM_CLASSES_PHASE2
    )

    # Val
    val_loss, val_iou, val_ious = validate(
        model_phase2, val_loader_phase2, criterion_phase2,
        DEVICE, NUM_CLASSES_PHASE2
    )

    # Scheduler
    scheduler_phase2.step(val_iou)
    current_lr = optimizer_phase2.param_groups[0]['lr']

    # Log
    history_phase2['train_loss'].append(train_loss)
    history_phase2['val_loss'].append(val_loss)
    history_phase2['val_iou'].append(val_iou)

    print(f"Train Loss: {train_loss:.4f} | Train IoU: {train_iou:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val IoU: {val_iou:.4f}")
    print(f"Val IoUs per class: {[f'{x:.3f}' for x in val_ious]}")
    print(f"LR: {current_lr}")

    # Save best
    if val_iou > best_iou_phase2:
        best_iou_phase2 = val_iou
        torch.save(model_phase2.state_dict(), 'galicia_finetuned.pth')
        print(f"✅ Guardado modelo con IoU: {best_iou_phase2:.4f}")

print(f"\n{'='*60}")
print(f"🎯 FASE 2 COMPLETADA - Mejor IoU: {best_iou_phase2:.4f}")
print(f"{'='*60}\n")

In [ ]:
# ==========================================
# TESTING FINAL FASE 2
# ==========================================
print("\n📊 TESTING FINAL (Galicia Fine-tuned)")
print("-" * 60)

# Cargar mejor modelo
model_phase2.load_state_dict(torch.load('galicia_finetuned.pth'))

test_loss, test_iou, test_ious = validate(
    model_phase2, val_loader_phase2, criterion_phase2,
    DEVICE, NUM_CLASSES_PHASE2
)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Mean IoU: {test_iou:.4f}")
print(f"Test IoU per class:")
class_names = ['Background', 'Pit', 'Scratch', 'Mechanical Damage']
for i, (name, iou) in enumerate(zip(class_names, test_ious)):
    print(f"  {name}: {iou:.4f}")

# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(history_phase2['train_loss'], label='Train Loss')
ax1.plot(history_phase2['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Galicia Fine-tuning Loss')
ax1.legend()
ax1.grid(True)

ax2.plot(history_phase2['val_iou'], label='Val IoU', color='green')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('IoU')
ax2.set_title('Galicia Fine-tuning IoU')
ax2.legend()
ax2.grid(True)
plt.tight_layout()
plt.show()

---
## 📊 RESUMEN FINAL
---

In [ ]:
print("\n" + "="*60)
print("📊 RESUMEN DE ENTRENAMIENTO EN 2 FASES")
print("="*60)

print(f"\n🔹 FASE 1 (Severstal Pre-training):")
print(f"   - Epochs: {PHASE1_EPOCHS}")
print(f"   - Best Val IoU: {best_iou_phase1:.4f}")
print(f"   - Modelo guardado: severstal_best.pth")

print(f"\n🔹 FASE 2 (Galicia Fine-tuning):")
print(f"   - Epochs: {PHASE2_EPOCHS}")
print(f"   - Best Val IoU: {best_iou_phase2:.4f}")
print(f"   - Test IoU: {test_iou:.4f}")
print(f"   - Modelo guardado: galicia_finetuned.pth")

print(f"\n🔹 Test IoU per class (Galicia):")
for name, iou in zip(class_names, test_ious):
    print(f"   - {name}: {iou:.4f}")

print("\n" + "="*60)
print("✅ ENTRENAMIENTO COMPLETADO")
print("="*60 + "\n")